In [4]:
# ========= USER CONFIG (required) =========
# INSERT YOUR FRED API KEY BELOW (do NOT commit this key to GitHub)
FRED_API_KEY = "fc2087ce43846d711d83874c0cacca79"   # <-- INSERT KEY HERE (e.g., "abcd1234...")

import os

if not FRED_API_KEY or FRED_API_KEY.strip() == "":
    raise RuntimeError(
        "FRED_API_KEY is empty.\n"
        "Insert your key in the first cell: FRED_API_KEY = '...'\n"
        "You can get it from https://fred.stlouisfed.org/ (API keys)."
    )

# Set it for this notebook session (no permanent storage)
os.environ["FRED_API_KEY"] = FRED_API_KEY.strip()

print("✅ FRED key loaded for this notebook session.")


✅ FRED key loaded for this notebook session.


In [5]:

from pathlib import Path
import os
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
SRC = PROJECT_ROOT / "src"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC:", SRC)
print("DATA_PROCESSED:", DATA_PROCESSED)

DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)


PROJECT_ROOT: C:\Users\rogow\PycharmProjects\JupyterProject
SRC: C:\Users\rogow\PycharmProjects\JupyterProject\src
DATA_PROCESSED: C:\Users\rogow\PycharmProjects\JupyterProject\data\processed


In [6]:
def run_py(script_name: str):
    script_path = SRC / script_name
    if not script_path.exists():
        raise FileNotFoundError(f"Brak pliku: {script_path}")
    print(f"\n=== RUN: {script_name} ===")
    res = subprocess.run(["python", str(script_path)], cwd=str(PROJECT_ROOT), capture_output=True, text=True)
    if res.stdout:
        print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
        raise RuntimeError(f"Skrypt {script_name} zakończył się błędem (code={res.returncode}).")
    if res.stderr:
        # ostrzeżenia itp.
        print(res.stderr)


In [7]:
# --- Data download ---
run_py("data_download.py")              # equities
run_py("data_download_fx.py")           # FX
run_py("data_download_bonds_fred.py")   # bonds (FRED)
run_py("data_download_commodities.py")  # commodities

# --- Skewness (12M rolling) ---
run_py("skewness.py")
run_py("skewness_fx.py")
run_py("skewness_bonds.py")
run_py("skewness_commodities.py")

# --- Portfolios (LS) ---
run_py("portfolios.py")
run_py("portfolios_fx.py")
run_py("portfolios_bonds.py")
run_py("portfolios_commodities.py")

# --- Performance summaries ---
run_py("analysis_equity.py")
run_py("analysis_fx.py")
run_py("analysis_bonds.py")
run_py("analysis_commodities.py")

# --- Regressions ---
run_py("regression_equity.py")
run_py("regression_bonds.py")



=== RUN: data_download.py ===
Pobieranie danych dla 10 tickerĂłw z yfinance...
Zapisano surowe dane equity do: C:\Users\rogow\PycharmProjects\JupyterProject\data\raw\equity_daily_prices.csv
Zapisano miesiÄ™czne stopy zwrotu equity do: C:\Users\rogow\PycharmProjects\JupyterProject\data\processed\equity_monthly_returns.csv
Pobieranie danych dla 12 tickerĂłw z yfinance...
Zapisano surowe dane 10Y yields do: C:\Users\rogow\PycharmProjects\JupyterProject\data\raw\rates_daily_yields.csv
Zapisano miesiÄ™czne zmiany rentownoĹ›ci do: C:\Users\rogow\PycharmProjects\JupyterProject\data\processed\rates_monthly_changes.csv

Podsumowanie:
Equity monthly returns shape: (239, 10)
Rates monthly changes shape: (0, 12)


[                       0%                       ]
[**********            20%                       ]  2 of 10 completed
[**************        30%                       ]  3 of 10 completed
[*******************   40%                       ]  4 of 10 completedHTTP Error 404: {"quoteSumm

RuntimeError: Skrypt skewness.py zakończył się błędem (code=1).